In [1]:
# Importing necessary libraries (see configuration file .yml in github repo)
#!pip install pybdshadow contextily folium pillow timezonefinder plotly

In [1]:
# Library Imports

import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
#from statsmodels.tsa.api import VAR
#from statsmodels.tsa.stattools import adfuller, grangercausalitytests
#from statsmodels.tools.eval_measures import rmse, aic
#from statsmodels.tsa.vector_ar.vecm import coint_johansen
#from statsmodels.stats.stattools import durbin_watson
import folium
import geopandas as gpd
from shapely.geometry import Polygon
import pytz
import pybdshadow
#from timezonefinder import TimezoneFinder
#import json
#import seaborn as sns
#from tabulate import tabulate
#import plotly.express as px
#from sklearn.mixture import GaussianMixture
#import pickle
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D,MaxPooling1D
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pybdshadow'

In [6]:
class ShadowModule:
        
        def __init__(self,lat, lon, roofs, spaces):
            self.lat= lat
            self.lon= lon
            self.roofs = roofs
            self.spaces= spaces
        
        
        def compute_coverage_rates(self, days_lst):
            """
            Compute coverage rates for each day in days_lst
            """

            # DataFrames to store results
            coverage_rates_df = pd.DataFrame()

            for current_date in days_lst:
                # Compute shadow projections for roofs
                shadows = self._all_sunshadeshadow_sunlight(current_date)

                # Calculate coverage rates
                coverage_rates = []
                for index, parking_space in self.spaces.iterrows():
                    parking_space_gdf = gpd.GeoDataFrame(geometry=[parking_space.geometry])
                    parking_space_gdf = parking_space_gdf.set_crs(epsg=4326)
                    parking_space_gdf = parking_space_gdf.to_crs(epsg=shadows.crs.to_epsg())

                    intersection = gpd.overlay(parking_space_gdf, shadows, how='intersection')

                    intersection_area = intersection.geometry.area.sum()
                    parking_space_area = parking_space_gdf.geometry.area.sum()

                    coverage_rate = intersection_area / parking_space_area
                    coverage_rates.append(coverage_rate)

                coverage_rates_df[f'coverage_rate_{current_date.strftime("%Y-%m-%d %H:%M:%S")}'] = coverage_rates

            return coverage_rates_df

            # Define function to calculate shadow and sunlight for all rooftops
        def _all_sunshadeshadow_sunlight(date):
            roof_projected_df= self.roofs
            roof_projected_df['geometry'] = roof_projected_df.apply(lambda r: self._sunshadeshadow_sunlight(date, r[0]), axis=1)
            return roof_projected_df
    
    
        def _sunshadeshadow_sunlight(date, r, sunshade_height=2):
            meanlon= r.centroid.x
            meanlat= r.centroid.y
            # obtain sun position
            sunPosition = get_position(date, meanlon, meanlat)
            if sunPosition['altitude'] < 0:
                raise ValueError("Given time before sunrise or after sunset")
                
            r_coords= np.array(r.exterior.coords)
            r_coords= r_coords.reshape(1,-1,2)
            shape = pybdshadow.utils.lonlat2aeqd(r_coords,meanlon,meanlat)
            azimuth = sunPosition['azimuth']
            altitude = sunPosition['altitude']

            n = np.shape(shape)[0]
            distance = sunshade_height / math.tan(altitude)

            # calculate the offset of the projection position
            lonDistance = distance * math.sin(azimuth)
            latDistance = distance * math.cos(azimuth)

            shadowShape = np.zeros((1, 5, 2))
            shadowShape[:, :, :] += shape
            shadowShape[:, :, 0] = shape[:, :, 0] + lonDistance
            shadowShape[:, :, 1] = shape[:, :, 1] + latDistance
            shadowShape = pybdshadow.utils.aeqd2lonlat(shadowShape,meanlon,meanlat)
            p = Polygon([[p[0], p[1]] for p in shadowShape[0]])
            return p


In [7]:
class DemandModule:
    def __init__(self, entry_exit_tuples, lr=1.0, show_details=True, refresh_model=False):

        print("Generating demand predictor...",end="")
        
        n_components = 2  
        self._gm = GaussianMixture(n_components=n_components, random_state=42)
        self._gm.fit(entry_exit_tuples[['enter_hour', 'exit_hour']])

        model_path = os.path.join('_models', 'demand_cnnlstm_model.keras')
        # Cargamos o entrenamos el modelo
        if os.path.exists(model_path) and not refresh_model:
            print(f"\n\tLoading model from {model_path}...", end="")
            model = load_model(model_path)
            print("DONE!")
            self._model
        else:
            print(f"\n\Training model...", end="")

            n_incoming_veh_df= entry_exit_tuples.groupby('date').size().reset_index()
            n_incoming_veh_df['datetime'] = pd.to_datetime(n_incoming_veh_df['date'])

            # Establecer 'datetime' como índice
            n_incoming_veh_df.set_index('datetime', inplace=True)

            # Renombrar la columna 'num_vehicles'
            n_incoming_veh_df.rename(columns={0: "num_vehicles"}, inplace=True)

            # Eliminar columnas originales si ya no se necesitan
            n_incoming_veh_df.drop(columns=["date"], inplace=True)

            scaler = MinMaxScaler()
            n_incoming_veh_df['num_vehicles'] = scaler.fit_transform(n_incoming_veh_df[['num_vehicles']])

            sequence_length = 12  # Longitud de las secuencias
            values = n_incoming_veh_df['num_vehicles'].values
            X, y = self._create_unidimensional_sequences(values, sequence_length)

            # Dividir los datos en conjunto de entrenamiento y prueba
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-lr, random_state=42)

            # Cambiar la forma de los datos para adaptarse a la entrada CNN-LSTM
            X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
            X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

            # Definir el modelo CNN-LSTM
            self._model = Sequential([
                Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
                MaxPooling1D(pool_size=2),
                LSTM(50, activation='relu', return_sequences=False),
                Dense(1)
            ])

            self._model.compile(optimizer='adam', loss='mse')

            _verbose= 0
            if show_details:
                _verbose= 1
            # Configuración de EarlyStopping
            early_stopping = EarlyStopping(
                monitor="val_loss",  # Métrica que se monitorea
                patience=10,         # Número de épocas de espera sin mejoras antes de detener
                restore_best_weights=True,  # Restaurar los mejores pesos
                verbose=_verbose          # Mostrar mensajes
            )

            # Entrenamiento con EarlyStopping
            self._model.fit(
                X_train,
                y_train,
                epochs=100,
                batch_size=16,
                validation_data=(X_test, y_test),
                callbacks=[early_stopping],  # Incluir el callback
                verbose=1
            )
            
            self._last_sequence = X_test[-1].flatten() 
            self._model.save(model_path)

            print("DONE!")

    # Crear los datos de entrada y salida para la serie temporal
    def _create_unidimensional_sequences(self, data, sequence_length):
        X, y = [], []
        for i in range(len(data) - sequence_length):
            X.append(data[i:i + sequence_length])
            y.append(data[i + sequence_length])
        return np.array(X), np.array(y)
    
    def _predict_future_occupancy(self, n_days_ahead):

        future_predictions = []
        current_sequence = self._last_sequence.copy()

        for _ in range(n_days_ahead*24):
            # Redimensionar la secuencia actual para que sea compatible con el modelo
            input_data = np.array(current_sequence).reshape((1, self.sequence_length, 1))
            
            # Predecir el siguiente valor
            next_pred = self._model.predict(input_data, verbose=0)[0][0]
            
            # Guardar el valor predicho (desnormalizado)
            future_predictions.append(self.scaler.inverse_transform([[next_pred]])[0][0])

            # Actualizar la secuencia de entrada con la nueva predicción
            current_sequence = np.append(current_sequence[1:], next_pred)

        return future_predictions

    def _generate_entry_and_exit_hours(self, n_tuples):
        new_samples, _ = gm.sample(n_samples=n_tuples)
        return [[round(x[0]),round(x[1])] for x in new_samples]
    
    def generate_occupancy(self, num_days_ahead):
        """
        Simulates vehicle parking occupancy and generates a table of occupancy information.
        """
        n_vehicles_per_hour = self._predict_future_occupancy(num_days_ahead)
        
        for n_vehicles in n_vehicles_per_hour: 
        
            new_samples, _ = gm.sample(n_samples=n_vehicles)
        

    

In [ ]:
class AmbientModule:
    
    def __init__(self, lat, lon, cabin_temperatures):
        self.lat= lat
        self.lon= lon


    def train_cabin_temperature_model(self, combined_temp_df, lr=1.0, show_details=False, refresh_model=False):
        """
        Trains a LSTM model using the combined temperature data.

        :param combined_temp_data: DataFrame containing combined temperature data
        """
        model_path = os.path.join('_models', 'cabintemp_lstm_model.keras')

        init_date =  self.cabin_temperatures.index[0].date()
        final_date =  self.cabin_temperatures.index[-1].date()

        # Normalización de los datos
        scaler_a = MinMaxScaler(feature_range=(0, 1))
        scaler_b = MinMaxScaler(feature_range=(0, 1))

        data = combined_temp_df.copy()
        data["temperature_2m"] = scaler_a.fit_transform(combined_temp_df["temperature_2m"].values.reshape(-1, 1))
        data["recorded_cabin_temp"] = scaler_b.fit_transform(combined_temp_df["recorded_cabin_temp"].values.reshape(-1, 1))

        look_back = 12  # Número de pasos anteriores a considerar
        X, y = self._create_sequences(data.values, look_back)

        # Dividimos en entrenamiento y prueba
        train_size = int(len(X) * lr)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        # Redimensionamos las entradas para LSTM [samples, time steps, features]
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        model = None
        # Cargamos o entrenamos el modelo
        if os.path.exists(model_path) and not refresh_model:
            print(f"\n\t Loading model from {model_path}...")
            model = load_model(model_path)
        else:
            print(f"\n\tTraining and saving model in {model_path}...")

            _verbose= 0
            if show_details:
              _verbose= 1
            # Configuración de EarlyStopping
            early_stopping = EarlyStopping(
                monitor="val_loss",  # Métrica que se monitorea
                patience=10,         # Número de épocas de espera sin mejoras antes de detener
                restore_best_weights=True,  # Restaurar los mejores pesos
                verbose=_verbose           # Mostrar mensajes
            )


            # Construcción del modelo LSTM
            model = Sequential()
            model.add(LSTM(50, input_shape=(look_back, 1)))
            model.add(Dense(1))
            model.compile(optimizer="adam", loss="mse")

            # Entrenamiento con EarlyStopping
            model.fit(
                X_train,
                y_train,
                epochs=1000,
                batch_size=16,
                validation_data=(X_test, y_test),
                callbacks=[early_stopping],  # Incluir el callback
                verbose=1
            )
        model.save(model_path)

        return model, scaler_a, scaler_b

    # Función para obtener datos horarios históricos de Open-Meteo API
    def _fetch_historical_temperature(self, start_date, end_date):
          """
          Obtiene datos horarios históricos de Open-Meteo API.

          Args:
          - latitude (float): Latitud de la ubicación.
          - longitude (float): Longitud de la ubicación.
          - start_date (str): Fecha de inicio en formato YYYY-MM-DD.
          - end_date (str): Fecha de fin en formato YYYY-MM-DD.
          - parameters (list): Variables meteorológicas a consultar, ej. ['temperature_2m', 'humidity_2m'].

          Returns:
          - pd.DataFrame: Datos meteorológicos horarios como DataFrame.
          """
          base_url = "https://archive-api.open-meteo.com/v1/archive"


          # Crear el payload para la solicitud
          payload = {
              "latitude": self.lat,
              "longitude": self.lon,
              "start_date": start_date,
              "end_date": end_date,
              "hourly": 'temperature_2m',
              "timezone": "auto"
          }

          # Realizar la solicitud a la API
          response = requests.get(base_url, params=payload)

          if response.status_code == 200:
              # Convertir la respuesta JSON a un DataFrame
              data = response.json()
              if "hourly" in data:
                  df = pd.DataFrame(data["hourly"])
                  df['time'] = pd.to_datetime(df['time'])
                  df= df.set_index('time')
                  return df
              else:
                  print("No se encontraron datos en la respuesta.")
                  return pd.DataFrame()
          else:
              print(f"Error en la solicitud: {response.status_code} - {response.text}")
              return pd.DataFrame()
          

    def _forecast_uncovered_cabin_temperatures(self, ambient_temp):
        ambient_temp_scaled = self.temp_scaler.fit_transform(ambient_temp)
        y_pred = self.cabin_temp_model.predict(ambient_temp_scaled)
        y_pred_rescaled = self.cabin_temp_scaler.inverse_transform(y_pred)
        return y_pred_rescaled

In [ ]:
class ClimaticPark:
    def __init__(self, file_name_lots='data/parking_lots.geojson',
                 file_name_roofs='data/parking_roofs.geojson',
                 file_name_coords='data/parking_coordinates.csv',
                 file_name_gates='data/gates_coordinates.csv',
                 file_name_mapcenter='data/map_center_coordinates.csv',
                 file_name_cabintem='data/historical_cabin_temp.csv',
                 file_name_tuples='data/entry_exit_tuples.csv'):
        """
        Initializes the ClimaticPark object by loading all necessary files.
        """
        # Load GeoJSON files for lots and roofs
        self.lots_data = self._load_geojson(file_name_lots)
        self.roofs_data = self._load_geojson(file_name_roofs)

        # Add 'height' column to roofs_data with a value of 1 for all rows
        if (self.lots_data is not None) and ('height' not in self.lots_data.columns):
            self.lots_data['height'] = 1  # Assuming a default height of 1

        # Add 'height' column to roofs_data with a value of 1 for all rows
        if (self.roofs_data is not None) and ('height' not in self.roofs_data.columns):
            self.roofs_data['height'] = 1  # Assuming a default height of 1

        # Load CSV files for coordinates, historical data, and additional data
        self.coords_data = self._load_csv(file_name_coords)
        self.gates_data = self._load_csv(file_name_gates)
      
        self.data_no_roof =  ClimaticPark.load_csv(os.path.join('data', 'cabin_temperature_no_roof.csv'))
        self.data_no_roof['coverage']=0
        self.data_roof =  ClimaticPark.load_csv(os.path.join('data', 'cabin_temperature_w_roof.csv'))
        self.data_roof['coverage']=1

        self.recorded_cabin_temp = ClimaticPark.read_csv(file_name_cabintem, index_col=0)
        # Convertir la columna DateTime a tipo datetime
        self.recorded_cabin_temp['DateTime'] = pd.to_datetime(self.recorded_cabin_temp['DateTime'])
        # Establecer la columna DateTime como índice
        self.recorded_cabin_temp.set_index('DateTime', inplace=True)
        # Remuestrear los datos para obtener una frecuencia de 1 hora (calculando la media)
        self.recorded_cabin_temp = self.recorded_cabin_temp.resample('h').mean()

        self.entry_exit_tuples= pd.read_csv(os.path.join('data', 'entry_exit_tuples_clean.csv'),
                                          index_col=0, dtype={'id_subject':str}, parse_dates=['date'])
        
        os.makedirs('_models', exist_ok=True)
        self.cabin_temp_model = None
        self.cabin_coverage_model= None
        self.demand_module = DemandModule(self.entry_exit_tuples)
        
        lat = self.coords_data['latitude'].iloc[0]  
        lon = self.coords_data['longitude'].iloc[0]
        self.shadow_module = ShadowModule(lat, lon, self.roofs_data, self.lots_data)

    @staticmethod
    def load_geojson(file_name):
        """
        Loads a GeoJSON file into a GeoDataFrame.
        """
        if file_name:
            return gpd.read_file(file_name)
        else:
            print(f"No GeoJSON file provided for {file_name}.")
            return None

    @staticmethod
    def load_csv(file_name):
        """
        Loads a CSV file into a DataFrame.
        """
        if file_name:
            return pd.read_csv(file_name)
        else:
            print(f"No CSV file provided for {file_name}.")
            return None

    def prepare_simulation(self, lr=0.8, display_details=False):
        if not self.coords_data.empty:
            latitude = self.coords_data['latitude'].iloc[0]  # Get the first coordinate
            longitude = self.coords_data['longitude'].iloc[0]
        else:
            raise ValueError("The coordinates file is empty or not formatted correctly.")

        #init_date =  self.recorded_cabin_temp.index[0].date()
        #final_date =  self.recorded_cabin_temp.index[-1].date()

        # Process temperature data
        print("Fetching historical weather conditions of the TPL...",end="")
        ambient_temp_df = self._fetch_historical_temperature(latitude, longitude, init_date, final_date)
        print("DONE!")
        combined_temp_df= pd.concat([ambient_temp_df, self.recorded_cabin_temp], axis=1)
        combined_temp_df = combined_temp_df.dropna()


        print("Training cabin temperature predictors...",end="")
        self.cabin_temp_model, self.temp_scaler, self.cabin_temp_scaler = self._train_cabin_temperature_model(combined_temp_df, lr, display_details)
        self.cabin_coverage_model= self._train_cabin_temperature_and_coverage_model()
        print("DONE!")

        print("Training demand predictor...",end="")
        self.demand_model = self._train_demand_model(lr)
        print("DONE!")

        print("Training GMM of entry-exit hours...",end="")
        self.gmm_demand= self._train_entry_exit_gmm()
        print("DONE!")

        print("Simulation ready to go!!")

    def launch_simulation(self, n_days_ahead, display_details=True):
        """
        Lauch simulation for n_days_ahead 
        """

        if not self.coords_data.empty:
            latitude = self.coords_data['latitude'].iloc[0]  # Get the first coordinate
            longitude = self.coords_data['longitude'].iloc[0]
        else:
            raise ValueError("The coordinates file is empty or not formatted correctly.")

        init_day = self.entry_exit_tuples.max('date').date()

        date_lst = [init_day + timedelta(days=i) for i in range(n_days_ahead)]
        days_lst = [date.date() for date in date_lst]

        ambient_temp= self._fetch_historical_temperature(self, latitude, longitude, init_day, days_lst[-1])
        ambient_temp= ambient_temp.loc[init_date_dt:final_date_dt]
        uncovered_cabin_temp= self._forecast_uncovered_cabin_temperatures(ambient_temp)
        
        coverage_rates= self.shadow_module.compute_coverage_rates(days_lst)




    # Preparamos el dataset para secuencias
    def _create_sequences(self, data, look_back):
        X, y = [], []
        for i in range(len(data) - look_back):
            X.append(data[i:i + look_back, 0])  # Secuencias de la variable 'a'
            y.append(data[i + look_back, 1])  # Predicción futura de la variable 'b'
        return np.array(X), np.array(y)


    

    def _train_cabin_temperature_and_coverage_model(self):

        temp_data = pd.concat([self.data_no_roof, self.data_roof], axis=0)

        X= temp_data['T temp_ext coverage'.split()].values
        y=  temp_data['temp_int'].values

        clf = LinearRegression().fit(X, y)
        return clf

In [16]:
park = ClimaticPark() #default parameters
park.prepare_simulation()
park.launch_simulation('2022-10-24 07:00:00')

Fetching historical weather conditions of the TPL...DONE!
Training cabin temperature predictor...Loading model from _models\cabintemp_lstm_model.keras...
DONE!
Generando modelo de temperatura de cabina y cover rates...DONE!
Generating demand predictor...
	Loading model from _models\demand_cnnlstm_model.keras...DONE!
Training gaussian mixture model of entry-exit hours...DONE!
Simulation ready to go!!


TypeError: 'method' object is not subscriptable

In [ ]:
raw_park_records_df= pd.read_csv(os.path.join('data', 'entry_exit_tuples_clean.csv'), index_col=0,
                              dtype={'id_subject':str}, parse_dates=['date'])
raw_park_records_df.head()

In [ ]:
init_day= raw_park_records_df['date'].max()
print(init_day)
n_days = 4
date_lst = [init_day + timedelta(days=i) for i in range(n_days)]
days_lst= [date.date() for date in date_lst]

print(days_lst)

In [ ]:
n_components = 2  # Número de componentes de la mezcla
gm = GaussianMixture(n_components=n_components, random_state=42)
gm.fit(raw_park_records_df[['enter_hour', 'exit_hour']])

new_samples, _ = gm.sample(n_samples=100)
[[round(x[0]),round(x[1])] for x in new_samples]

In [ ]:
n_incoming_veh_df= raw_park_records_df.groupby('date').size().reset_index()
# Crear columna datetime a partir de 'date' y 'enter_hour'
n_incoming_veh_df['datetime'] = pd.to_datetime(n_incoming_veh_df['date'])

# Establecer 'datetime' como índice
n_incoming_veh_df.set_index('datetime', inplace=True)

# Renombrar la columna 'num_vehicles'
n_incoming_veh_df.rename(columns={0: "num_vehicles"}, inplace=True)

# Eliminar columnas originales si ya no se necesitan
n_incoming_veh_df.drop(columns=["date"], inplace=True)
n_incoming_veh_df

In [ ]:
ax=n_incoming_veh_df.plot(figsize=(15,5), grid=True)
ax.set_xlabel('Date',fontsize=20);
ax.set_ylabel('Num. of hourly users',fontsize=20);
ax.tick_params(axis='y', labelsize=15)
ax.tick_params(axis='x', labelsize=15)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
n_incoming_veh_df['num_vehicles'] = scaler.fit_transform(n_incoming_veh_df[['num_vehicles']])

# Crear los datos de entrada y salida para la serie temporal
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 12  # Longitud de las secuencias
values = n_incoming_veh_df['num_vehicles'].values
X, y = create_sequences(values, sequence_length)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cambiar la forma de los datos para adaptarse a la entrada CNN-LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Definir el modelo CNN-LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(50, activation='relu', return_sequences=False),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Configuración de EarlyStopping
early_stopping = EarlyStopping(
    monitor="val_loss",  # Métrica que se monitorea
    patience=10,         # Número de épocas de espera sin mejoras antes de detener
    restore_best_weights=True,  # Restaurar los mejores pesos
    verbose=1          # Mostrar mensajes
)

# Entrenar el modelo
#history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Entrenamiento con EarlyStopping
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],  # Incluir el callback
    verbose=1
)


# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print(f"Pérdida en el conjunto de prueba: {loss}")

# Hacer predicciones
y_pred = model.predict(X_test)

# Desescalar los resultados para obtener valores originales
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_rescaled = scaler.inverse_transform(y_pred)

print(f"Valores originales predichos: {y_pred_rescaled[:5].flatten()}")

In [ ]:
import requests
import pandas as pd

# Función para obtener datos horarios históricos de Open-Meteo API
def get_historical_weather_hourly(latitude, longitude, start_date, end_date):
    """
    Obtiene datos horarios históricos de Open-Meteo API.

    Args:
    - latitude (float): Latitud de la ubicación.
    - longitude (float): Longitud de la ubicación.
    - start_date (str): Fecha de inicio en formato YYYY-MM-DD.
    - end_date (str): Fecha de fin en formato YYYY-MM-DD.
    - parameters (list): Variables meteorológicas a consultar, ej. ['temperature_2m', 'humidity_2m'].

    Returns:
    - pd.DataFrame: Datos meteorológicos horarios como DataFrame.
    """
    base_url = "https://archive-api.open-meteo.com/v1/archive"


    # Crear el payload para la solicitud
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": 'temperature_2m',
        "timezone": "auto"
    }

    # Realizar la solicitud a la API
    response = requests.get(base_url, params=payload)

    if response.status_code == 200:
        # Convertir la respuesta JSON a un DataFrame
        data = response.json()
        if "hourly" in data:
            df = pd.DataFrame(data["hourly"])

            return df
        else:
            print("No se encontraron datos en la respuesta.")
            return pd.DataFrame()
    else:
        print(f"Error en la solicitud: {response.status_code} - {response.text}")
        return pd.DataFrame()

ambient_temperatures_df = get_historical_weather_hourly(park.coords_data['latitude'].iloc[0], park.coords_data['longitude'].iloc[0], init_date, final_date)
ambient_temperatures_df

In [ ]:
ambient_temperatures_df['time'] = pd.to_datetime(ambient_temperatures_df['time'])

ambient_temperatures_df= ambient_temperatures_df.set_index('time')
ambient_temperatures_df